In [ ]:
# python version 3.6.9 #
import pandas as pd # 1.1.5
import numpy as np # 1.19.5
import matplotlib.pyplot as plt
import matplotlib # 3.3.4
import scipy.stats as st # scipy 1.5.4

matplotlib.rcParams['font.sans-serif'] = "Arial"
matplotlib.rcParams['font.family'] = "sans-serif"
plt.rc('text', usetex=False)
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams.update({'font.size': 16})
plt.style.use('source_data/included/figure.style')


In [ ]:
###### CALM DATA ######
combined = pd.read_csv('source_data/included/fig4_CRISPRi_enrichment.txt',sep='\t',index_col=0)
for rRNA_type in ['rrf','rrl','rrs']:
    LAG = combined.loc[(combined['LAGs'].str.contains(rRNA_type)) & (combined['LAGs'].str.contains('b'))].iloc[0,0]
    to_append = combined.loc[(combined['LAGs'].str.contains(rRNA_type)) & (combined['LAGs'].str.contains('b'))].groupby('strand').mean().reset_index()
    to_append.loc[:,to_append.columns.str.contains('signif')] = np.round(to_append.loc[:,to_append.columns.str.contains('signif')])
    to_append['LAGs'] = [LAG,LAG]
    combined = combined.loc[~((combined['LAGs'].str.contains(rRNA_type)) & (combined['LAGs'].str.contains('b')))]
    combined = combined.append(to_append)
    combined = combined.reset_index().drop('index',axis=1)

## define replicates for each sample type
metG_lag = ['SBC210_vs_SBC205','SBC101_vs_SBC96','SBC102_vs_SBC96','SBC100_vs_SBC96','SBC192_vs_SBC191']
MG1655_lag = ['SBC131_vs_SBC126','SBC132_vs_SBC126','SBC130_vs_SBC126']
metG_lag_abx = ['SBC207_vs_SBC210','SBC208_vs_SBC210','SBC209_vs_SBC210','SBC98_vs_SBC101','SBC99_vs_SBC101']
MG1655_lag_abx = ['SBC128_vs_SBC131','SBC129_vs_SBC131']
metG_exp = ['SBC212_vs_SBC211']
MG1655_exp = ['SBC134_vs_SBC133']
hipA7_lag = ['SBC318_vs_SBC316']
hipA7_lag_abx = ['SBC317_vs_SBC318']

combined = combined.loc[combined.loc[:,combined.columns.str.contains('n_positions')].sum(axis=1)>0]

combined['gene_ID'] = combined['LAGs'].str.split('=',expand=True)[0]
combined['gene_name'] = combined['LAGs'].str.split('=',expand=True)[1]
combined = combined.drop('LAGs',axis=1)

combined = combined.loc[:,combined.columns[[-2,-1]].append(combined.columns[0:-2])]

combined = combined.sort_values('strand').sort_values('gene_ID')


pval_table = combined.loc[:,combined.columns.str.contains('p_val|gene|strand')]
n_positions_table = combined.loc[:,combined.columns.str.contains('n_positions|gene|strand')]

pval_table.columns = pval_table.columns.str.replace('p_val_','')
n_positions_table.columns = n_positions_table.columns.str.replace('n_positions_','')

pval_stack = pval_table.stack()
pval_stack[n_positions_table.stack()==0] = 1
pval_table = pval_stack.unstack()

pval_table.to_csv('tables/TableS5_pval.csv',sep=',')

temp = combined.loc[:,combined.columns.str.contains('enrichment|gene|strand')]
temp.columns = temp.columns.str.replace('enrichment_','')
temp.to_csv('tables/TableS5_enrichment.csv',sep=',')

temp = combined.loc[:,combined.columns.str.contains('n_positions|gene|strand')]
temp.columns = temp.columns.str.replace('n_positions_','')
temp.to_csv('tables/TableS5_nPositions.csv',sep=',')

temp = combined.loc[:,combined.columns.str.contains('signif|gene|strand')]
temp.columns = temp.columns.str.replace('signif_','')
temp.to_csv('tables/TableS5_signif.csv',sep=',')

